# Real estate price model workflow

Sam Maurer, Feb 2018; Paul Waddell, June 2018

Python 3.6, intended to be backward compatible with 2.7

In [2]:
from __future__ import print_function
import warnings;warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

In [3]:
# Standard to run UrbanSim from the root level of the project directory

import os; os.chdir('../..')

In [4]:
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import OLSRegressionStep
import orca

In [5]:
# Load any script-based Orca registrations
from scripts import datasources
from scripts import models

## Explore the Orca registrations

In [6]:
orca.list_tables()

['parcels', 'buildings', 'rentals', 'units', 'households', 'persons', 'jobs']

In [ ]:
for table_name in orca.list_tables():
    print(table_name.upper())
    print(orca.get_table(table_name).to_frame().columns.tolist())
    print()

In [9]:
orca.list_broadcasts()

[('parcels', 'buildings'),
 ('buildings', 'units'),
 ('units', 'households'),
 ('households', 'persons'),
 ('nodes', 'parcels'),
 ('nodes', 'rentals'),
 ('nodes', 'units')]

In [ ]:
orca.list_steps()

## Explore the data

Orca doesn't execute code to load the registered objects until it needs to.

In [12]:
orca.get_table('parcels').to_frame().describe()

,primary_id,development_type_id,land_value,acres,county_id,zone_id,proportion_undevelopable,tax_exempt_status,geom_id,x,y,shape_area,block_id,node_id
count,1.956207e+06,1.956207e+06,1.956207e+06,1.956207e+06,1.956207e+06,1.956207e+06,1.956207e+06,1.956207e+06,1.956207e+06,1.956207e+06,1.956207e+06,1.956207e+06,1.956207e+06,1.956207e+06
mean,1.018502e+06,2.607759e+00,2.348019e+05,2.294288e+00,5.393547e+01,8.293200e+02,1.633774e-03,1.256104e-02,8.795114e+12,-1.221763e+02,3.777351e+01,9.284664e+03,6.054294e+13,1.234190e+05
std,5.972833e+05,5.310605e+00,1.610710e+06,3.612166e+01,3.808734e+01,4.129778e+02,4.038695e-02,1.113700e-01,5.076215e+12,2.991972e-01,3.621875e-01,1.461793e+05,3.807600e+11,1.031260e+05
min,1.000000e+00,-1.000000e+00,0.000000e+00,4.527090e-07,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.706688e+07,-1.235266e+02,3.689751e+01,1.832050e-03,6.001400e+13,0.000000e+00
25%,4.999735e+05,1.000000e+00,3.995700e+04,1.084042e-01,1.300000e+01,4.920000e+02,0.000000e+00,0.000000e+00,4.405538e+12,-1.224059e+02,3.748103e+01,4.386967e+02,6.013321e+13,4.811400e+04
50%,1.005872e+06,1.000000e+00,1.008000e+05,1.489650e-01,7.500000e+01,8.440000e+02,0.000000e+00,0.000000e+00,8.793873e+12,-1.221419e+02,3.775390e+01,6.028407e+02,6.075033e+13,9.553400e+04
75%,1.544652e+06,1.000000e+00,2.254345e+05,2.294452e-01,8.500000e+01,1.189000e+03,0.000000e+00,0.000000e+00,1.318756e+13,-1.219512e+02,3.798583e+01,9.285328e+02,6.085508e+13,1.434330e+05
max,2.054502e+06,2.500000e+01,1.393464e+09,2.335199e+04,9.700000e+01,1.454000e+03,1.000000e+00,1.000000e+00,1.759217e+13,-1.212109e+02,3.886017e+01,9.450224e+07,6.113012e+13,4.157150e+05


In [13]:
orca.get_table('buildings').to_frame().describe()

,building_id,parcel_id,development_type_id,improvement_value,residential_units,residential_sqft,sqft_per_unit,non_residential_sqft,building_sqft,nonres_rent_per_sqft,res_price_per_sqft,stories,year_built,redfin_sale_price,redfin_sale_year,building_type_id,node_id
count,1.824229e+06,1.824229e+06,1.824229e+06,1.824229e+06,1.824229e+06,1.824229e+06,1.824229e+06,1.824229e+06,1.824229e+06,1.824229e+06,1.824229e+06,1.824229e+06,1.824229e+06,1.081100e+06,1.081100e+06,1.824229e+06,1.748191e+06
mean,9.223790e+05,1.012861e+06,1.835162e+00,3.186878e+05,1.534000e+00,2.657113e+03,1.669294e+03,1.280864e+03,3.784462e+03,9.218572e-01,2.927191e+02,1.214429e+00,1.965524e+03,5.287307e+05,2.002828e+03,1.710299e+00,1.234532e+05
std,5.317352e+05,5.938065e+05,3.441080e+00,3.571671e+06,6.137263e+00,3.012408e+04,1.063476e+03,2.392226e+04,3.494568e+04,4.053916e+00,2.647712e+02,5.567798e-01,2.501754e+01,7.188123e+05,7.143867e+00,2.396463e+00,1.030772e+05
min,1.000000e+00,2.600000e+01,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.790000e+03,0.000000e+00,1.962000e+03,0.000000e+00,2.000000e+00
25%,4.623410e+05,4.958500e+05,1.000000e+00,8.267503e+04,1.000000e+00,1.230000e+03,1.176000e+03,0.000000e+00,1.320000e+03,0.000000e+00,1.518671e+02,1.000000e+00,1.952000e+03,2.390000e+05,1.998000e+03,1.000000e+00,4.828050e+04
50%,9.224730e+05,1.002691e+06,1.000000e+00,1.675535e+05,1.000000e+00,1.754000e+03,1.564000e+03,0.000000e+00,1.858000e+03,0.000000e+00,2.544594e+02,1.000000e+00,1.966000e+03,4.000000e+05,2.004000e+03,1.000000e+00,9.563600e+04
75%,1.382840e+06,1.532117e+06,1.000000e+00,2.853423e+05,1.000000e+00,2.400000e+03,2.029424e+03,0.000000e+00,2.524000e+03,0.000000e+00,3.699121e+02,1.000000e+00,1.983000e+03,6.500000e+05,2.009000e+03,1.000000e+00,1.434330e+05
max,1.843350e+06,2.054501e+06,2.400000e+01,3.355514e+09,1.912000e+03,1.458000e+07,3.000000e+04,1.657310e+07,1.458000e+07,1.289625e+02,5.016109e+04,5.900000e+01,2.015000e+03,1.800000e+08,2.014000e+03,1.600000e+01,4.157150e+05


## Generate accessibility measures for the price model

The network accessibility metrics are not stored on disk; for now we'll generate them using legacy code.

In [7]:
orca.run(['initialize_network'])

Running step 'initialize_network'
Time to execute step 'initialize_network': 11.97 s
Total time to execute iteration 1 with iteration value None: 11.97 s


In [10]:
orca.run(['network_aggregations'])

Running step 'network_aggregations'
Computing accessibility variables
Computing residential_units_500
Removed 76038 rows because they contain missing values
Computing residential_units_1500
Removed 76038 rows because they contain missing values
Computing population_1500
Removed 305829 rows because they contain missing values
Computing poor_1500
Removed 79602 rows because they contain missing values
Computing renters_1500
Removed 145852 rows because they contain missing values
Computing med_income_1500
Removed 305829 rows because they contain missing values
Computing job_1500
Removed 111044 rows because they contain missing values
Computing job_25km
Removed 111044 rows because they contain missing values
Computing ave_hhsize
Removed 305829 rows because they contain missing values
Computing med_rent_sqft_1500
Computing med_rent_sqft_7000
       residential_units_500  residential_units_1500  population_1500  \
count          415716.000000           415716.000000    415716.000000   
mean  

In [ ]:
orca.list_tables()

In [15]:
print(orca.get_table('nodes').to_frame().columns.tolist())

['residential_units_500', 'residential_units_1500', 'population_1500', 'poor_1500', 'renters_1500', 'med_income_1500', 'job_1500', 'job_25km', 'ave_hhsize', 'med_rent_sqft_1500', 'med_rent_sqft_7000']


# Estimate a price model

The basic idea of the parcel template is that we create model steps by _passing arguments to classes_ rather than by writing Python functions and giving them Orca decorators, as we would for a fully custom model.

Much of the functionality for this is already built into UrbanSim and Orca, we'll just need to extend things here and there.

This demo uses a new RegressionStep() class that provides a full model development workflow: estimating a model, registering it with Orca, saving it for future use.

### Specify parameters and pass them to a model object

In [22]:
m = OLSRegressionStep()
m.tables = ['buildings', 'parcels', 'nodes']
m.filters = 'redfin_sale_year > 2010 & residential_units == 1'
m.model_expression = 'np.log1p(res_price_per_sqft) ~ year_built + residential_units_500 + renters_1500 + \
    job_1500'
m.fit()

                                 OLS Regression Results                                 
Dep. Variable:     np.log1p(res_price_per_sqft)   R-squared:                       0.057
Model:                                      OLS   Adj. R-squared:                  0.057
Method:                           Least Squares   F-statistic:                     2316.
Date:                          Sun, 08 Jul 2018   Prob (F-statistic):               0.00
Time:                                  15:01:53   Log-Likelihood:            -1.9407e+05
No. Observations:                        153926   AIC:                         3.882e+05
Df Residuals:                            153921   BIC:                         3.882e+05
Df Model:                                     4                                         
Covariance Type:                      nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------

### If we like it, register it as an Orca step

In [ ]:
model.register()

In [ ]:
orca.list_steps()

### Run the Orca step

In [ ]:
orca.run(['RegressionStep-20180214-210159'])

### Check the fitted values

In [ ]:
df = orca.get_table('buildings').to_frame(['res_price_per_sqft', 'fitted_price'])
df.loc[df.fitted_price > 0].describe()

### BONUS

Running "model.register()" also registered the step with the new ModelManager extention, which saves it to disk so that it can be automatically re-loaded in the future.

The "test" model steps here were estimated earlier and loaded from disk. They're fully functional: we can run them in Orca, inspect the estimation results, etc.

In [ ]:
mm.list_steps()

In [ ]:
rs = mm.get_step('test-1')
type(rs)

In [ ]:
rs.model_expression

In [ ]:
rs.model.report_fit()